In [1]:
import pandas as pd
import swifter

info = "info_CUB.csv"
df = pd.read_csv(info, sep=";")

def read_text(path):
    with open(path, "r") as f:
        text = f.read()
    return text

df["text"] = df["text"].swifter.apply(read_text)

label2id = {k: v for v, k in enumerate(df["label"].unique())}
id2label = {v: k for k, v in label2id.items()}
df["label"] = df["label"].map(label2id)

columns = ["image", "text", "label"]
data_train = df[df["train"] == 1][columns]
data_test = df[df["train"] == 0][columns]

Pandas Apply:   0%|          | 0/11788 [00:00<?, ?it/s]

In [2]:
# Get text from the model
from transformers import AutoModel, AutoTokenizer, AutoFeatureExtractor
from PIL import Image

# Load models gpu 0
model_nlp = AutoModel.from_pretrained("texts_model")
tokenizer_nlp = AutoTokenizer.from_pretrained("texts_model")

# Get images from the model
feature_extractor_img = AutoFeatureExtractor.from_pretrained("images_model")
model_img = AutoModel.from_pretrained("images_model")




2022-11-25 01:28:39.798133: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-25 01:28:39.798159: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Some weights of the model checkpoint at texts_model were not used when initializing MPNetModel: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias']
- This IS expected if you are initializing MPNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MPNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification m

In [3]:
import numpy as np
def get_image_vector(image, model=model_img, feature_extractor=feature_extractor_img):
    # Get last layer of the model
    image = Image.open(image)
    inputs = feature_extractor(images=image, return_tensors="pt")
    outputs = model(**inputs)
    embeddings = outputs.pooler_output.detach().numpy()
    return embeddings


def get_text_vector(text, model=model_nlp, tokenizer=tokenizer_nlp):
    #return np.random.rand(100)
    inputs = tokenizer_nlp(text, return_tensors="pt")
    outputs = model_nlp(**inputs)
    embeddings = outputs.pooler_output.detach().numpy()
    return embeddings

data_train = data_train.sample(100)
data_train["v_text"] = data_train["text"].apply(get_text_vector)
data_train["v_image"] = data_train["image"].apply(get_image_vector)

# def get_data_vector(image,text):
#     try:
#         image_v = get_image_vector(image)
#         text_v = get_text_vector(text)
#         # Join two torch.Tensor in one
#         v =  np.concatenate((image_v, text_v), axis=1)
#         # Tranpose to get the right shape
#         return v
#     except:
#         return np.nan

# data_train["vector"], da = data_train.apply(lambda x: get_data_vector(x["image"], x["text"]), axis=1)
# data_test["vector"] = data_test.apply(lambda x: get_data_vector(x["image"], x["text"]), axis=1)

data_train.to_pickle("vectors_swin_sbert/data_train.pkl")
data_test.to_pickle("vectors_swin_sbert/data_test.pkl")

In [31]:
# Join two vectors in one
print(data_train["v_image"][2920].T.shape)
print(data_train["v_text"][2920].T.shape)

(768, 1)
(768, 1)


In [32]:
data_train["vector"] = data_train.apply(lambda x: np.concatenate((x["v_image"].T, x["v_text"].T), axis=1), axis=1)

In [2]:
import pandas as pd 
data_train = pd.read_pickle("vectors_swin_sbert/data_train.pkl")
data_test = pd.read_pickle("vectors_swin_sbert/data_test.pkl")

data_train = data_train.dropna().reset_index(drop=True)
data_test = data_test.dropna().reset_index(drop=True)

In [3]:
# Convert vector to a list of numbers


data_train["vector"] = data_train["vector"].apply(lambda x: x.tolist()[0])
data_test["vector"] = data_test["vector"].apply(lambda x: x.tolist()[0])

In [7]:
# MLP sklearn model with 4 hidden layers
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

X_train = data_train["vector"].tolist()
y_train = data_train["label"].tolist()

X_test = data_test["vector"].tolist()
y_test = data_test["label"].tolist()




In [21]:
# Grid search to find the best parameters for the model
from sklearn.model_selection import GridSearchCV

params = {
    "hidden_layer_sizes": [(1000, 300), (1000, 800, 500),(1000, 500)],
    "activation": ["tanh", "relu"],
    "solver": ["adam", "sgd"],
    "alpha": [0.05, 0.01],
    "learning_rate": ["adaptive"],
    
}

mlp = MLPClassifier(max_iter=20, random_state=42, verbose=False, early_stopping=True, validation_fraction=0.1)

# Custom scorer    

clf = GridSearchCV(mlp, params, n_jobs=4, verbose=10, cv=2, scoring="accuracy")) 
clf.fit(X_train, y_train)

print(clf.best_params_)

Fitting 2 folds for each of 24 candidates, totalling 48 fits
[CV 1/2; 1/24] START activation=tanh, alpha=0.05, hidden_layer_sizes=(1000, 300), learning_rate=adaptive, solver=adam
[CV 2/2; 1/24] START activation=tanh, alpha=0.05, hidden_layer_sizes=(1000, 300), learning_rate=adaptive, solver=adam
[CV 1/2; 2/24] START activation=tanh, alpha=0.05, hidden_layer_sizes=(1000, 300), learning_rate=adaptive, solver=sgd
[CV 2/2; 2/24] START activation=tanh, alpha=0.05, hidden_layer_sizes=(1000, 300), learning_rate=adaptive, solver=sgd
[CV 2/2; 1/24] END activation=tanh, alpha=0.05, hidden_layer_sizes=(1000, 300), learning_rate=adaptive, solver=adam;, score=0.954 total time=  28.5s
[CV 1/2; 3/24] START activation=tanh, alpha=0.05, hidden_layer_sizes=(1000, 800, 500), learning_rate=adaptive, solver=adam


/home/jlortiz/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV 1/2; 2/24] END activation=tanh, alpha=0.05, hidden_layer_sizes=(1000, 300), learning_rate=adaptive, solver=sgd;, score=0.835 total time=  38.6s
[CV 1/2; 1/24] END activation=tanh, alpha=0.05, hidden_layer_sizes=(1000, 300), learning_rate=adaptive, solver=adam;, score=0.960 total time=  39.0s


/home/jlortiz/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV 2/2; 3/24] START activation=tanh, alpha=0.05, hidden_layer_sizes=(1000, 800, 500), learning_rate=adaptive, solver=adam
[CV 2/2; 2/24] END activation=tanh, alpha=0.05, hidden_layer_sizes=(1000, 300), learning_rate=adaptive, solver=sgd;, score=0.846 total time=  39.0s
[CV 1/2; 4/24] START activation=tanh, alpha=0.05, hidden_layer_sizes=(1000, 800, 500), learning_rate=adaptive, solver=sgd
[CV 2/2; 4/24] START activation=tanh, alpha=0.05, hidden_layer_sizes=(1000, 800, 500), learning_rate=adaptive, solver=sgd
[CV 1/2; 3/24] END activation=tanh, alpha=0.05, hidden_layer_sizes=(1000, 800, 500), learning_rate=adaptive, solver=adam;, score=0.961 total time=  56.3s
[CV 1/2; 5/24] START activation=tanh, alpha=0.05, hidden_layer_sizes=(1000, 500), learning_rate=adaptive, solver=adam
[CV 2/2; 3/24] END activation=tanh, alpha=0.05, hidden_layer_sizes=(1000, 800, 500), learning_rate=adaptive, solver=adam;, score=0.963 total time=  55.4s


...

[CV 1/2; 24/24] END activation=relu, alpha=0.01, hidden_layer_sizes=(1000, 500), learning_rate=adaptive, solver=sgd;, score=0.898 total time=  41.1s
[CV 2/2; 23/24] END activation=relu, alpha=0.01, hidden_layer_sizes=(1000, 500), learning_rate=adaptive, solver=adam;, score=0.955 total time=  44.7s


/home/jlortiz/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV 1/2; 23/24] END activation=relu, alpha=0.01, hidden_layer_sizes=(1000, 500), learning_rate=adaptive, solver=adam;, score=0.947 total time=  48.5s


/home/jlortiz/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV 2/2; 24/24] END activation=relu, alpha=0.01, hidden_layer_sizes=(1000, 500), learning_rate=adaptive, solver=sgd;, score=0.904 total time=  37.8s
{'activation': 'tanh', 'alpha': 0.05, 'hidden_layer_sizes': (1000, 500), 'learning_rate': 'adaptive', 'solver': 'adam'}


In [22]:
print(clf.best_params_)
mlp = MLPClassifier(**clf.best_params_, max_iter=20, random_state=42, verbose=False, early_stopping=True, validation_fraction=0.1)
mlp.fit(X_train, y_train)
y_pred = mlp.predict(X_test)
print("Accuracy: ", accuracy_score(y_test, y_pred))


{'activation': 'tanh', 'alpha': 0.05, 'hidden_layer_sizes': (1000, 500), 'learning_rate': 'adaptive', 'solver': 'adam'}
Accuracy:  0.8142734307824592
